In [7]:
import pyshark

# Open saved trace file 
filename = 'home_startup_20220119a.pcap'
cap = pyshark.FileCapture(filename)

In [1]:
import pyshark
filename = 'day1__2022-04-13T07_45_37.pcap'
cap = pyshark.FileCapture(filename)

In [2]:
print(cap[7565])

Packet (Length: 174)
Layer ETH:
	Destination: 2c:97:b1:70:bb:06
	Address: 2c:97:b1:70:bb:06
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Source: c0:56:27:77:02:af
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Type: IPv4 (0x0800)
	Address: c0:56:27:77:02:af
Layer IP:
	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 160
	Identification: 0xfc2f (64559)
	Flags: 0x40, Don't fragment
	0... .... = Reserved bit: Not set
	.1.. .... = Don't fragment: Set
	..0. .... = More fragments: Not set
	Fragment Offset: 0
	Time to Live: 64
	Protocol: TCP (6)
	Header C

## Format:

### Timestamp;src_ip:src_port;dst_ip:dst_port;SNI;ALPN;Total_size_bytes;Src_size_bytes;Dst_size_bytes;Duration_secs

In [2]:
# importing the datetime package  
import datetime 
ls = []
i = 1
for pkt in cap:
    if ("TLS" in pkt):
        try:
            with open('file.txt', mode='w') as file_object:
                print(pkt.tls.handshake, file=file_object)
        except:
            continue
        with open('file.txt', mode='r') as file_object:
            string = file_object.read().splitlines()
        string = string[0].split(': ')[1]
        if string == 'Client Hello':
            timestamp = pkt.frame_info.time_epoch
            with open('file.txt', mode='w') as file_object:
                print(timestamp, file=file_object)
            with open('file.txt', mode='r') as file_object:
                string = file_object.read().splitlines()
            date_time = datetime.datetime.fromtimestamp(float(string[0]))
            src = f'{pkt.ip.src}:{pkt.tcp.srcport}'
            dst = f'{pkt.ip.dst}:{pkt.tcp.dstport}'
            try:
                sni = pkt.tls.handshake_extensions_server_name
            except:
                sni = 'NA'
            try:
                alpn = pkt.tls.handshake_extensions_alpn_str
            except:
                alpn = 'NA'
            ls.append([date_time,src,dst,sni,alpn])
    i+=1
print(ls)

[[datetime.datetime(2022, 1, 19, 22, 54, 6, 157982), '192.168.1.73:37164', '8.8.8.8:853', 'NA', 'NA'], [datetime.datetime(2022, 1, 19, 22, 54, 6, 158117), '192.168.1.73:37746', '8.8.4.4:853', 'NA', 'NA'], [datetime.datetime(2022, 1, 19, 22, 54, 7, 187283), '192.168.1.73:41118', '74.125.68.103:443', 'www.google.com', 'http/1.1'], [datetime.datetime(2022, 1, 19, 22, 54, 7, 331162), '192.168.1.73:38492', '142.250.4.188:5228', 'mtalk.google.com', 'NA'], [datetime.datetime(2022, 1, 19, 22, 54, 7, 517194), '192.168.1.73:37756', '8.8.4.4:853', 'NA', 'NA'], [datetime.datetime(2022, 1, 19, 22, 54, 7, 607697), '192.168.1.73:40538', '162.247.243.189:443', 'mobile-collector.newrelic.com', 'http/1.1'], [datetime.datetime(2022, 1, 19, 22, 54, 8, 911210), '192.168.1.73:45414', '74.125.200.155:443', 'googleads.g.doubleclick.net', 'NA'], [datetime.datetime(2022, 1, 19, 22, 54, 9, 1213), '192.168.1.73:48228', '157.240.13.14:443', 'graph.facebook.com', 'http/1.1'], [datetime.datetime(2022, 1, 19, 22, 54,

In [3]:
import pandas as pd
df = pd.DataFrame(ls,columns=['Timestamp','Src','Dst','SNI','ALPN'])
df

,Timestamp,Src,Dst,SNI,ALPN
0,2022-01-19 22:54:06.157982,192.168.1.73:37164,8.8.8.8:853,NA,NA
1,2022-01-19 22:54:06.158117,192.168.1.73:37746,8.8.4.4:853,NA,NA
2,2022-01-19 22:54:07.187283,192.168.1.73:41118,74.125.68.103:443,www.google.com,http/1.1
3,2022-01-19 22:54:07.331162,192.168.1.73:38492,142.250.4.188:5228,mtalk.google.com,NA
4,2022-01-19 22:54:07.517194,192.168.1.73:37756,8.8.4.4:853,NA,NA
...,...,...,...,...,...
228,2022-01-19 22:55:56.419019,192.168.1.73:48156,157.240.15.1:443,graph.facebook.com,http/1.1
229,2022-01-19 22:55:56.422803,192.168.1.73:48158,157.240.15.1:443,graph.facebook.com,http/1.1
230,2022-01-19 22:55:56.775194,192.168.1.73:37178,34.102.215.127:443,global.glbtls.t.co,http/1.1
231,2022-01-19 22:55:57.359229,192.168.1.73:48588,35.201.69.248:443,na.glbtls.t.co,h2


In [4]:
df.to_csv('sample.csv',index=False)

In [5]:
print(cap[164])
float(cap[164].frame_info.time_epoch)

Packet (Length: 583)
Layer ETH:
	Destination: 2c:97:b1:70:bb:06
	Address: 2c:97:b1:70:bb:06
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Source: 0e:7e:63:80:54:08
	.... ..1. .... .... .... .... = LG bit: Locally administered address (this is NOT the factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Type: IPv4 (0x0800)
	Address: 0e:7e:63:80:54:08
Layer IP:
	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 569
	Identification: 0x8f0e (36622)
	Flags: 0x40, Don't fragment
	0... .... = Reserved bit: Not set
	.1.. .... = Don't fragment: Set
	..0. .... = More fragments: Not set
	Fragment Offset: 0
	Time to Live: 64
	Protoc

1642604047.187283

In [6]:
#print(cap[82].ip.src,cap[82].ip.dst)
#print(cap[82].tcp.srcport,cap[82].tcp.dstport)
print(dir(cap[164].tls))
print(cap[164].tls.handshake_length)

['', 'DATA_LAYER', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_all_fields', '_field_prefix', '_get_all_field_lines', '_get_all_fields_with_alternates', '_get_field_or_layer_repr', '_get_field_repr', '_layer_name', '_sanitize_field_name', 'extension_psk_ke_mode', 'extension_psk_ke_modes_length', 'field_names', 'get', 'get_field', 'get_field_by_showname', 'get_field_value', 'handshake', 'handshake_cipher_suites_length', 'handshake_ciphersuite', 'handshake_ciphersuites', 'handshake_comp_method', 'handshake_comp_methods', 'handshake_comp_methods_length', 'handshake_extension_data', 'handshake_extension_len', 'handshake_extension_type', 'handshake_

In [7]:
print(cap[164].tls.handshake)
print(type(cap[164].tls))
with open('file.txt', mode='w') as file_object:
    print(cap[164].tls.handshake, file=file_object)
with open('file.txt', mode='r') as file_object:
    string = file_object.read().splitlines()
    print(string[0])
    print(type(string))

string = string[0].split(': ')[1]
print(string)
print(list(string))
print(string=='Client Hello')
print(cap[164].frame_info.time_epoch)

Handshake Protocol: Client Hello
<class 'pyshark.packet.layer.Layer'>
Handshake Protocol: Client Hello
<class 'list'>
Client Hello
['C', 'l', 'i', 'e', 'n', 't', ' ', 'H', 'e', 'l', 'l', 'o']
True
1642604047.187283000


In [8]:
# importing the datetime package  
import datetime  
  
# given epoch time  
epoch_time = 1642603747.457011  
  
# using the datetime.fromtimestamp() function  
date_time = datetime.datetime.fromtimestamp( epoch_time )  
  
# printing the value  
print("Given epoch time:", epoch_time)  
print("Converted Datetime:", date_time )  

Given epoch time: 1642603747.457011
Converted Datetime: 2022-01-19 22:49:07.457011


## (Not Needed & can be removed) NFStream implementation to get flow bytes and flow duration

In [ ]:
'''
Jupyter notebook has issues with getting the dataframe out from NFStream, dataframe is obtained via Python script &
is converted to csv file for data analysis here
'''
import pandas as pd
from nfstream import NFStreamer, NFPlugin

class FlowSlicer(NFPlugin):
    def on_init(self, packet, flow):
        if self.limit == 1:
           flow.expiration_id = -1 

    def on_update(self, packet, flow):
        if self.limit == flow.bidirectional_packets:
           flow.expiration_id = -1 # -1 value force expiration
print(filename)
def get_dataframe():
    my_dataframe = NFStreamer(source=filename,decode_tunnels=True,
                             bpf_filter=None,
                             promiscuous_mode=True,
                             snapshot_length=1536,
                             idle_timeout=120,
                             active_timeout=1800,
                             accounting_mode=2,
                             udps=FlowSlicer(limit=7),
                             n_dissections=20,
                             statistical_analysis=True,
                             splt_analysis=0,
                             n_meters=0,
                             performance_report=0,
                             system_visibility_mode=0,
                             system_visibility_poll_ms=100,
                             system_visibility_extension_port=28314).to_pandas()
    return my_dataframe



if __name__ == '__main__':
    df = get_dataframe()
    print(df)

home_startup_20220119a.pcap


In [ ]:
import pandas as pd
df1 = pd.read_csv('NFStream.csv')
pd.set_option('display.max_columns', None)
df1

In [ ]:
def filter_TLS(tls):
    if 'TLS' in tls:
        return True
    else:
        return False
df1['Boolean'] = df1['application_name'].apply(filter_TLS)
df1

In [ ]:
df2 = df1[df1['Boolean']==True]
df2

In [ ]:
df2[(df2['src_ip']=='192.168.1.73') & (df2['src_port']==41118) & (df2['dst_ip']=='74.125.68.103') & (df2['dst_port']==443)]


In [ ]:
print(cap[164].frame_info.time_epoch)

## Extend Pyshark to get the stream information

In [9]:
print(cap[164])

Packet (Length: 583)
Layer ETH:
	Destination: 2c:97:b1:70:bb:06
	Address: 2c:97:b1:70:bb:06
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Source: 0e:7e:63:80:54:08
	.... ..1. .... .... .... .... = LG bit: Locally administered address (this is NOT the factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Type: IPv4 (0x0800)
	Address: 0e:7e:63:80:54:08
Layer IP:
	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 569
	Identification: 0x8f0e (36622)
	Flags: 0x40, Don't fragment
	0... .... = Reserved bit: Not set
	.1.. .... = Don't fragment: Set
	..0. .... = More fragments: Not set
	Fragment Offset: 0
	Time to Live: 64
	Protoc

In [10]:
print(dir(cap[164].tcp))

['', 'DATA_LAYER', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_all_fields', '_field_prefix', '_get_all_field_lines', '_get_all_fields_with_alternates', '_get_field_or_layer_repr', '_get_field_repr', '_layer_name', '_sanitize_field_name', 'ack', 'ack_raw', 'analysis', 'analysis_bytes_in_flight', 'analysis_initial_rtt', 'analysis_push_bytes_sent', 'checksum', 'checksum_status', 'dstport', 'field_names', 'flags', 'flags_ack', 'flags_cwr', 'flags_ecn', 'flags_fin', 'flags_ns', 'flags_push', 'flags_res', 'flags_reset', 'flags_str', 'flags_syn', 'flags_urg', 'get', 'get_field', 'get_field_by_showname', 'get_field_value', 'hdr_len', 'layer_name', 'le

In [44]:
print(cap[164].tcp.stream)
print(type(cap[164].tcp.stream))
with open('file.txt', mode='w') as file_object:
    print(cap[164].tcp.stream, file=file_object)
with open('file.txt', mode='r') as file_object:
    string = file_object.read().splitlines()
print(string)
print(type(string[0]))

11
<class 'pyshark.packet.fields.LayerFieldsContainer'>
['11']
<class 'str'>


In [50]:
i=0
for pkt in cap:
   i+=1
print(i)

19927


In [87]:
# importing the datetime package  
import datetime
ls = []
stream_list = []
i = 1
for pkt in cap:
    if ("TLS" in pkt):
        try:
            with open('file.txt', mode='w') as file_object:
                print(pkt.tls.handshake, file=file_object)
        except:
            continue
        with open('file.txt', mode='r') as file_object:
            string = file_object.read().splitlines()
        string = string[0].split(': ')[1]
        if string == 'Client Hello':
            timestamp = pkt.frame_info.time_epoch
            string = Pyshark_Converter(timestamp)
            date_time = float(string)
            src = f'{pkt.ip.src}:{pkt.tcp.srcport}'
            dst = f'{pkt.ip.dst}:{pkt.tcp.dstport}'
            
            stream = pkt.tcp.stream
            if stream in stream_list:
                continue
            stream_list.append(stream)
            byte = int(Pyshark_Converter(pkt.tcp.len))
            ls.append([date_time,src,dst,stream, byte])
            continue
    if ("TCP" in pkt):
        timestamp = pkt.frame_info.time_epoch
        string = Pyshark_Converter(timestamp)
        date_time = float(string)
        src = f'{pkt.ip.src}:{pkt.tcp.srcport}'
        dst = f'{pkt.ip.dst}:{pkt.tcp.dstport}'

        stream = pkt.tcp.stream
        if stream not in stream_list:
            continue
        #stream_list.append(stream)
        byte = int(Pyshark_Converter(pkt.tcp.len))
        ls.append([date_time,src,dst,stream, byte])


In [88]:
df = pd.DataFrame(ls,columns=['Timestamp','Src','Dst','Stream_ID','Bytes'])
df

,Timestamp,Src,Dst,SNI,ALPN,Stream_ID,Bytes
0,1.642604e+09,192.168.1.73:37164,8.8.8.8:853,NA,NA,9,230
1,1.642604e+09,192.168.1.73:37746,8.8.4.4:853,NA,NA,8,230
2,1.642604e+09,8.8.8.8:853,192.168.1.73:37164,NA,NA,9,0
3,1.642604e+09,8.8.4.4:853,192.168.1.73:37746,NA,NA,8,0
4,1.642604e+09,8.8.8.8:853,192.168.1.73:37164,NA,NA,9,2836
...,...,...,...,...,...,...,...
6944,1.642604e+09,192.168.1.73:48588,35.201.69.248:443,NA,NA,281,0
6945,1.642604e+09,192.168.1.73:48588,35.201.69.248:443,NA,NA,281,0
6946,1.642604e+09,192.168.1.73:48588,35.201.69.248:443,NA,NA,281,0
6947,1.642604e+09,35.201.69.248:443,192.168.1.73:48588,NA,NA,281,0


In [89]:
df.to_csv('reference.csv',index=False)

In [90]:
df['Timestamp'].value_counts()

1.642604e+09    4
1.642604e+09    4
1.642604e+09    4
1.642604e+09    4
1.642604e+09    4
               ..
1.642604e+09    1
1.642604e+09    1
1.642604e+09    1
1.642604e+09    1
1.642604e+09    1
Name: Timestamp, Length: 6706, dtype: int64

In [91]:
df[df['Stream_ID'] == '9']

,Timestamp,Src,Dst,SNI,ALPN,Stream_ID,Bytes
0,1.642604e+09,192.168.1.73:37164,8.8.8.8:853,NA,NA,9,230
2,1.642604e+09,8.8.8.8:853,192.168.1.73:37164,NA,NA,9,0
4,1.642604e+09,8.8.8.8:853,192.168.1.73:37164,NA,NA,9,2836
5,1.642604e+09,8.8.8.8:853,192.168.1.73:37164,NA,NA,9,1418
7,1.642604e+09,192.168.1.73:37164,8.8.8.8:853,NA,NA,9,0
8,1.642604e+09,192.168.1.73:37164,8.8.8.8:853,NA,NA,9,0
9,1.642604e+09,192.168.1.73:37164,8.8.8.8:853,NA,NA,9,0
10,1.642604e+09,192.168.1.73:37164,8.8.8.8:853,NA,NA,9,0
18,1.642604e+09,8.8.8.8:853,192.168.1.73:37164,NA,NA,9,0
19,1.642604e+09,8.8.8.8:853,192.168.1.73:37164,NA,NA,9,612


In [103]:
concat = []
client_hello = pd.read_csv('sample.csv')
for row in client_hello.itertuples(index=True, name='Pandas'):
    src = row.Src
    stream_id = row.Stream_ID
    #print(stream_id)
    #print(type(stream_id))
    timestamp = row.Timestamp
    temp_df = df[df['Stream_ID']==str(stream_id)]
    total_bytes, src_bytes, dst_bytes, duration_s = Pyshark_Byte_Duration_Extender(temp_df, src)
    concat.append([total_bytes, src_bytes, dst_bytes, duration_s])
    
conc = pd.DataFrame(concat,columns=['Total_size_bytes','Src_size_bytes','Dst_size_bytes','Duration_secs'])
conc

,Total_size_bytes,Src_size_bytes,Dst_size_bytes,Duration_secs
0,5096,230,4866,0.876593
1,3685,294,3391,0.885287
2,3353,517,2836,0.263096
3,6189,517,5672,54.031336
4,3066,230,2836,108.562394
...,...,...,...,...
226,1897,517,1380,0.319019
227,1897,517,1380,0.385313
228,1935,517,1418,0.355253
229,3353,517,2836,0.451489


In [104]:
final_product = pd.concat([client_hello, conc], axis=1)
final_product

,Timestamp,Src,Dst,SNI,ALPN,Stream_ID,Total_size_bytes,Src_size_bytes,Dst_size_bytes,Duration_secs
0,2022-01-19 22:54:06.157982,192.168.1.73:37164,8.8.8.8:853,NaN,NaN,9,5096,230,4866,0.876593
1,2022-01-19 22:54:06.158117,192.168.1.73:37746,8.8.4.4:853,NaN,NaN,8,3685,294,3391,0.885287
2,2022-01-19 22:54:07.187283,192.168.1.73:41118,74.125.68.103:443,www.google.com,http/1.1,11,3353,517,2836,0.263096
3,2022-01-19 22:54:07.331162,192.168.1.73:38492,142.250.4.188:5228,mtalk.google.com,NaN,12,6189,517,5672,54.031336
4,2022-01-19 22:54:07.517194,192.168.1.73:37756,8.8.4.4:853,NaN,NaN,13,3066,230,2836,108.562394
...,...,...,...,...,...,...,...,...,...,...
226,2022-01-19 22:55:56.419019,192.168.1.73:48156,157.240.15.1:443,graph.facebook.com,http/1.1,278,1897,517,1380,0.319019
227,2022-01-19 22:55:56.422803,192.168.1.73:48158,157.240.15.1:443,graph.facebook.com,http/1.1,279,1897,517,1380,0.385313
228,2022-01-19 22:55:56.775194,192.168.1.73:37178,34.102.215.127:443,global.glbtls.t.co,http/1.1,280,1935,517,1418,0.355253
229,2022-01-19 22:55:57.359229,192.168.1.73:48588,35.201.69.248:443,na.glbtls.t.co,h2,281,3353,517,2836,0.451489


In [101]:
'''
Convert pyshark.packet.fields.LayerFieldsContainer into string format
'''
def Pyshark_Converter(item):


    with open('file.txt', mode='w') as file_object:
        print(item, file=file_object)
    with open('file.txt', mode='r') as file_object:
        string = file_object.read().splitlines()
    return string[0]
'''
Get duration & src2dst bytes & dst2src bytes using given dataframe
'''
def Pyshark_Byte_Duration_Extender(df, src_client):    

    duration = []
    src_byte = []
    dst_byte = []
    for row in df.itertuples(index=True, name='Pandas'):
        duration.append(row.Timestamp)
        src = row.Src
        if src == src_client:
            src_byte.append(row.Bytes)
        else:
            dst_byte.append(row.Bytes)
    duration_s = max(duration) - min(duration)
    src_bytes = sum(src_byte)
    dst_bytes = sum(dst_byte)
    total_bytes = src_bytes + dst_bytes
    return total_bytes, src_bytes, dst_bytes, duration_s 

#print(f'Stream Duration is {duration_s}')
#print(f'Src bytes is {src_bytes}')
#print(f'Dst bytes is {dst_bytes}')
#print(f'Total bytes is {total_bytes}')

Stream Duration is 0.7357409000396729

Src bytes is 839

Dst bytes is 5147

Total bytes is 5986

In [84]:
duration_s, src_bytes, dst_bytes, total_bytes = Pyshark_Byte_Duration_Extender(cap, '9', '192.168.1.73')
print(f'Stream Duration is {duration_s}')
print(f'Src bytes is {src_bytes}')
print(f'Dst bytes is {dst_bytes}')
print(f'Total bytes is {total_bytes}')

[1642604046.148133, 1642604046.151085, 1642604046.157682, 1642604046.157982, 1642604046.162062, 1642604046.163497, 1642604046.163543, 1642604046.163672, 1642604046.164499, 1642604046.177788, 1642604046.177789, 1642604046.17779, 1642604046.714618, 1642604046.714639, 1642604046.717567, 1642604046.71976, 1642604046.934606, 1642604047.024306, 1642604047.027671, 1642604047.027803, 1642604047.0306, 1642604047.031264, 1642604047.034575]
Stream Duration is 0.8864419460296631
Src bytes is 396
Dst bytes is 6033
Total bytes is 6429


In [ ]:
ls = []
stream_list = []
i = 1
for pkt in cap:
    if ("TLS" in pkt):
        try:
            with open('file.txt', mode='w') as file_object:
                print(pkt.tls.handshake, file=file_object)
        except:
            continue
        with open('file.txt', mode='r') as file_object:
            string = file_object.read().splitlines()
        string = string[0].split(': ')[1]
        if string == 'Client Hello':
            timestamp = pkt.frame_info.time_epoch
            string = Pyshark_Converter(timestamp)
            date_time = datetime.datetime.fromtimestamp(float(string[0]))
            src = f'{pkt.ip.src}:{pkt.tcp.srcport}'
            dst = f'{pkt.ip.dst}:{pkt.tcp.dstport}'
            try:
                sni = pkt.tls.handshake_extensions_server_name
            except:
                sni = 'NA'
            try:
                alpn = pkt.tls.handshake_extensions_alpn_str
            except:
                alpn = 'NA'
            stream = pkt.tcp.stream
            if stream in stream_list:
                continue
            stream_list.append(stream)
            stream_id = Pyshark_Converter(stream)
            duration, src_b, dst_b, total_b = Pyshark_Byte_Duration_Extender(cap,stream_id,f'{pkt.ip.src}')
            ls.append([date_time,src,dst,sni,alpn,total_b,src_b,dst_b,duration])
            i+=1
            print(f'Iteration: {i}')

In [ ]:
df = pd.DataFrame(ls,columns=['Timestamp','Src','Dst','SNI','ALPN','Total_Bytes','Src2Dst_Bytes','Dst2Src_Bytes','Duration_s'])
df

In [4]:
import time
start_time = time.time()
time.sleep(5)
print(time.time() - start_time)

5.007097959518433


In [6]:
import pandas as pd
df = pd.read_csv('phone1_startup_yt_Sin-map_dbs.csv')
df

,Timestamp,Src,Dst,SNI,ALPN,Stream_ID,Total_size_bytes,Src_size_bytes,Dst_size_bytes,Duration_secs
0,2022-03-16 21:36:23.544553,192.168.1.91:49087,8.8.4.4:853,NaN,NaN,0,5114,93,5021,0.152213
1,2022-03-16 21:36:23.544859,192.168.1.91:39659,8.8.8.8:853,NaN,NaN,1,5114,93,5021,0.152208
2,2022-03-16 21:36:23.642366,192.168.1.91:43319,142.251.10.103:443,www.google.com,http/1.1,2,2836,0,2836,0.189409
3,2022-03-16 21:36:23.650903,192.168.1.91:45066,74.125.200.188:5228,mtalk.google.com,NaN,4,16822,0,16822,4.092520
4,2022-03-16 21:36:23.800833,192.168.1.91:49097,8.8.4.4:853,NaN,NaN,5,5613,93,5520,66.511329
...,...,...,...,...,...,...,...,...,...,...
433,2022-03-16 22:41:28.601329,192.168.1.91:39178,8.8.4.4:853,NaN,NaN,0,5113,93,5020,0.058944
434,2022-03-16 22:41:28.669328,192.168.1.91:39182,8.8.4.4:853,NaN,NaN,3,5114,93,5021,5.420270
435,2022-03-16 22:41:28.721580,192.168.1.91:40551,142.251.12.105:443,www.google.com,http/1.1,4,218,0,218,0.066670
436,2022-03-16 22:41:28.738102,192.168.1.91:48495,74.125.200.188:5228,mtalk.google.com,NaN,5,203,0,203,0.091440


In [8]:
df1 = pd.read_csv('Activities_Day1-20220413.csv')
df1

,Timestamp,Src,Dst,SNI,ALPN,Stream_ID,Total_size_bytes,Src_size_bytes,Dst_size_bytes,Duration_secs
0,2022-04-13 07:47:01.531239,192.168.1.125:48992,52.9.236.142:443,cloud.linksyssmartwifi.com,NaN,10,5052,0,5052,0.576318
1,2022-04-13 07:47:18.210746,192.168.1.127:40474,18.140.10.111:443,mp-sg-rcdda-3.auto.mydlink.com,NaN,11,11483402,11478245,5157,198.121319
2,2022-04-13 07:48:11.588978,192.168.1.82:47093,54.151.136.73:443,mp-sg-rcdda-7.auto.mydlink.com,NaN,13,5601154,5595997,5157,144.507550
3,2022-04-13 07:50:39.112441,192.168.1.89:54390,18.140.10.111:443,mp-sg-rcdda-3.auto.mydlink.com,NaN,4,12453318,12448161,5157,98.874959
4,2022-04-13 08:16:56.113570,192.168.1.89:53990,54.151.136.73:443,mp-sg-rcdda-7.auto.mydlink.com,NaN,7,10206659,10201502,5157,73.984953
...,...,...,...,...,...,...,...,...,...,...
2191,2022-04-13 11:04:38.039820,192.168.1.96:58832,40.70.161.102:443,in.appcenter.ms,http/1.1,138,5071,0,5071,2.690667
2192,2022-04-13 11:04:40.506821,192.168.1.98:53672,52.168.112.67:443,teams.events.data.microsoft.com,h2,139,4765,1,4764,45.947689
2193,2022-04-13 11:05:22.593982,192.168.1.78:38938,8.8.4.4:853,NaN,NaN,140,203,0,203,14.254479
2194,2022-04-13 11:05:22.890651,192.168.1.78:38058,205.239.42.27:443,appsentry.eu.baesystemsai.com,NaN,141,4082,1183,2899,13.181111


In [9]:
filename = 'phone1_startup_yt_Sin-map_dbs/home_phonestart_2022-03-16T22_41_22.pcap'
cap = pyshark.FileCapture(filename)

In [20]:
def Pyshark_Converter(item):

    with open('file.txt', mode='w') as file_object:
        print(item, file=file_object)
    with open('file.txt', mode='r') as file_object:
        string = file_object.read().splitlines()
    return string[0]

'''
Get duration & src2dst bytes & dst2src bytes using given dataframe
'''
def Pyshark_Byte_Duration_Extender(df, src_client):    

    duration = []
    src_byte = []
    dst_byte = []
    for row in df.itertuples(index=True, name='Pandas'):
        duration.append(row.Timestamp)
        src = row.Src
        if src == src_client:
            src_byte.append(row.Bytes)
        else:
            dst_byte.append(row.Bytes)
    duration_s = max(duration) - min(duration)
    src_bytes = sum(src_byte)
    dst_bytes = sum(dst_byte)
    total_bytes = src_bytes + dst_bytes
    return total_bytes, src_bytes, dst_bytes, duration_s 

In [78]:
 
import datetime
ls = []
ref = []
stream_list = []
#debug = []
i = 1
for pkt in cap:
    #debug.append(i)
    #if i == 143:
    #    print(pkt)
    if ("TLS" in pkt):
        try:
            with open('file.txt', mode='w') as file_object:
                print(pkt.tls.handshake, file=file_object)
        except:
            with open('file.txt', mode='w') as file_object:
                print("[': ']", file=file_object)
        with open('file.txt', mode='r') as file_object:
            string = file_object.read().splitlines()
            #print(string)
        string = string[0].split(': ')[1]
        timestamp = pkt.frame_info.time_epoch
        times = Pyshark_Converter(timestamp)
        date_time = datetime.datetime.fromtimestamp(float(times))
        src = f'{pkt.ip.src}:{pkt.tcp.srcport}'
        dst = f'{pkt.ip.dst}:{pkt.tcp.dstport}'
        stream = pkt.tcp.stream
        
        if string == 'Client Hello':
            try:
                sni = pkt.tls.handshake_extensions_server_name
            except:
                sni = 'NA'
            try:
                alpn = pkt.tls.handshake_extensions_alpn_str
            except:
                alpn = 'NA'
            if stream in stream_list:
                #print("In stream list")
                #i+=1
                continue
            stream_list.append(stream)
            ls.append([date_time,src,dst,sni,alpn,stream])
        byte = int(Pyshark_Converter(pkt.tcp.len))
        ref.append([float(times),src,dst,stream, byte])
        #i+=1
        continue

    if ("TCP" in pkt):
        timestamp = pkt.frame_info.time_epoch
        string = Pyshark_Converter(timestamp)
        date_time = float(string)
        src = f'{pkt.ip.src}:{pkt.tcp.srcport}'
        dst = f'{pkt.ip.dst}:{pkt.tcp.dstport}'

        stream = pkt.tcp.stream
        #if stream not in stream_list:
        #    continue

        byte = int(Pyshark_Converter(pkt.tcp.len))
        ref.append([date_time,src,dst,stream, byte])
    #i+=1
#print(debug)

Packet (Length: 130)
Layer ETH:
	Destination: 2c:97:b1:70:bb:06
	Address: 2c:97:b1:70:bb:06
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Source: 92:44:a2:35:66:1e
	.... ..1. .... .... .... .... = LG bit: Locally administered address (this is NOT the factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Type: IPv4 (0x0800)
	Address: 92:44:a2:35:66:1e
Layer IP:
	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 116
	Identification: 0x81a6 (33190)
	Flags: 0x40, Don't fragment
	0... .... = Reserved bit: Not set
	.1.. .... = Don't fragment: Set
	..0. .... = More fragments: Not set
	Fragment Offset: 0
	Time to Live: 64
	Protoc

In [45]:
ls

[[datetime.datetime(2022, 3, 16, 22, 41, 28, 600811),
  '192.168.1.91:44513',
  '8.8.8.8:853',
  'NA',
  'NA',
  '1'],
 [datetime.datetime(2022, 3, 16, 22, 41, 28, 601329),
  '192.168.1.91:39178',
  '8.8.4.4:853',
  'NA',
  'NA',
  '0'],
 [datetime.datetime(2022, 3, 16, 22, 41, 28, 669328),
  '192.168.1.91:39182',
  '8.8.4.4:853',
  'NA',
  'NA',
  '3'],
 [datetime.datetime(2022, 3, 16, 22, 41, 28, 721580),
  '192.168.1.91:40551',
  '142.251.12.105:443',
  'www.google.com',
  'http/1.1',
  '4'],
 [datetime.datetime(2022, 3, 16, 22, 41, 28, 738102),
  '192.168.1.91:48495',
  '74.125.200.188:5228',
  'mtalk.google.com',
  'NA',
  '5'],
 [datetime.datetime(2022, 3, 16, 22, 41, 34, 122479),
  '192.168.1.91:46955',
  '74.125.130.95:443',
  'userlocation.googleapis.com',
  'h2',
  '6']]

In [46]:
ref

[[1647441688.592556, '192.168.1.91:39178', '8.8.4.4:853', '0', 0],
 [1647441688.592557, '192.168.1.91:44513', '8.8.8.8:853', '1', 0],
 [1647441688.59557, '8.8.8.8:853', '192.168.1.91:44513', '1', 0],
 [1647441688.595956, '8.8.4.4:853', '192.168.1.91:39178', '0', 0],
 [1647441688.600416, '192.168.1.91:39178', '8.8.4.4:853', '0', 0],
 [1647441688.600417, '192.168.1.91:44513', '8.8.8.8:853', '1', 0],
 [1647441688.600811, '192.168.1.91:44513', '8.8.8.8:853', '1', 135],
 [1647441688.600811, '192.168.1.91:44513', '8.8.8.8:853', '1', 135],
 [1647441688.601329, '192.168.1.91:39178', '8.8.4.4:853', '0', 135],
 [1647441688.601329, '192.168.1.91:39178', '8.8.4.4:853', '0', 135],
 [1647441688.603764, '8.8.8.8:853', '192.168.1.91:44513', '1', 0],
 [1647441688.604132, '8.8.4.4:853', '192.168.1.91:39178', '0', 0],
 [1647441688.607196, '8.8.8.8:853', '192.168.1.91:44513', '1', 1418],
 [1647441688.608375, '8.8.8.8:853', '192.168.1.91:44513', '1', 1418],
 [1647441688.609584, '8.8.8.8:853', '192.168.1.91

In [79]:
final_product = pd.DataFrame()
client_hello_df = pd.DataFrame(ls,columns=['Timestamp','Src','Dst','SNI','ALPN','Stream_ID'])
ref_df = pd.DataFrame(ref,columns=['Timestamp','Src','Dst','Stream_ID','Bytes'])
concat = []
        
for row in client_hello_df.itertuples(index=True, name='Pandas'):
    src = row.Src
    stream_id = row.Stream_ID
    timestamp = row.Timestamp
    temp_df = ref_df[ref_df['Stream_ID']==str(stream_id)]
    total_bytes, src_bytes, dst_bytes, duration_s = Pyshark_Byte_Duration_Extender(temp_df, src)
    concat.append([total_bytes, src_bytes, dst_bytes, duration_s])

conc = pd.DataFrame(concat,columns=['Total_size_bytes','Src_size_bytes','Dst_size_bytes','Duration_secs'])
        
intermediate_product = pd.concat([client_hello_df, conc], axis=1)
final_product = pd.concat([final_product, intermediate_product],axis=0,ignore_index=True)


In [80]:
client_hello_df

,Timestamp,Src,Dst,SNI,ALPN,Stream_ID
0,2022-03-16 22:41:28.600811,192.168.1.91:44513,8.8.8.8:853,NA,NA,1
1,2022-03-16 22:41:28.601329,192.168.1.91:39178,8.8.4.4:853,NA,NA,0
2,2022-03-16 22:41:28.669328,192.168.1.91:39182,8.8.4.4:853,NA,NA,3
3,2022-03-16 22:41:28.721580,192.168.1.91:40551,142.251.12.105:443,www.google.com,http/1.1,4
4,2022-03-16 22:41:28.738102,192.168.1.91:48495,74.125.200.188:5228,mtalk.google.com,NA,5
5,2022-03-16 22:41:34.122479,192.168.1.91:46955,74.125.130.95:443,userlocation.googleapis.com,h2,6


In [81]:
ref_df

,Timestamp,Src,Dst,Stream_ID,Bytes
0,1.647442e+09,192.168.1.91:39178,8.8.4.4:853,0,0
1,1.647442e+09,192.168.1.91:44513,8.8.8.8:853,1,0
2,1.647442e+09,8.8.8.8:853,192.168.1.91:44513,1,0
3,1.647442e+09,8.8.4.4:853,192.168.1.91:39178,0,0
4,1.647442e+09,192.168.1.91:39178,8.8.4.4:853,0,0
...,...,...,...,...,...
119,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,61
120,1.647442e+09,74.125.130.95:443,192.168.1.91:46955,6,31
121,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,0
122,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,31


In [83]:
final_product

,Timestamp,Src,Dst,SNI,ALPN,Stream_ID,Total_size_bytes,Src_size_bytes,Dst_size_bytes,Duration_secs
0,2022-03-16 22:41:28.600811,192.168.1.91:44513,8.8.8.8:853,NA,NA,1,5477,344,5133,0.058942
1,2022-03-16 22:41:28.601329,192.168.1.91:39178,8.8.4.4:853,NA,NA,0,5477,344,5133,0.058944
2,2022-03-16 22:41:28.669328,192.168.1.91:39182,8.8.4.4:853,NA,NA,3,6565,546,6019,5.420270
3,2022-03-16 22:41:28.721580,192.168.1.91:40551,142.251.12.105:443,www.google.com,http/1.1,4,1959,886,1073,0.066670
4,2022-03-16 22:41:28.738102,192.168.1.91:48495,74.125.200.188:5228,mtalk.google.com,NA,5,1805,955,850,0.091440
5,2022-03-16 22:41:34.122479,192.168.1.91:46955,74.125.130.95:443,userlocation.googleapis.com,h2,6,1568,717,851,0.069146


In [84]:
ref_df[ref_df['Stream_ID']==str(6)]

,Timestamp,Src,Dst,Stream_ID,Bytes
110,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,0
111,1.647442e+09,74.125.130.95:443,192.168.1.91:46955,6,0
112,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,0
113,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,561
114,1.647442e+09,74.125.130.95:443,192.168.1.91:46955,6,0
115,1.647442e+09,74.125.130.95:443,192.168.1.91:46955,6,212
116,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,0
117,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,64
118,1.647442e+09,74.125.130.95:443,192.168.1.91:46955,6,608
119,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,61


In [64]:
ref_df[ref_df['Stream_ID']==str(6)]

,Timestamp,Src,Dst,Stream_ID,Bytes
91,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,0
92,1.647442e+09,74.125.130.95:443,192.168.1.91:46955,6,0
93,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,0
94,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,561
95,1.647442e+09,74.125.130.95:443,192.168.1.91:46955,6,0
96,1.647442e+09,74.125.130.95:443,192.168.1.91:46955,6,212
97,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,0
98,1.647442e+09,192.168.1.91:46955,74.125.130.95:443,6,0
99,1.647442e+09,74.125.130.95:443,192.168.1.91:46955,6,0


In [43]:
i = 0
for pkt in cap:
    i+=1
    #print(pkt)
print(f'No.of packets: {i}')

No.of packets: 151


In [70]:
print(cap[142])

Packet (Length: 130)
Layer ETH:
	Destination: 2c:97:b1:70:bb:06
	Address: 2c:97:b1:70:bb:06
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Source: 92:44:a2:35:66:1e
	.... ..1. .... .... .... .... = LG bit: Locally administered address (this is NOT the factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Type: IPv4 (0x0800)
	Address: 92:44:a2:35:66:1e
Layer IP:
	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 116
	Identification: 0x81a6 (33190)
	Flags: 0x40, Don't fragment
	0... .... = Reserved bit: Not set
	.1.. .... = Don't fragment: Set
	..0. .... = More fragments: Not set
	Fragment Offset: 0
	Time to Live: 64
	Protoc